In [2]:
import trimesh
import pycork
import numpy as np
import os
from pathlib import Path
from typing import Union
import pyvista 
import pyacvd
import subprocess

Problems:
Нету функции для merge by vertices, поэтому у нас требование, чтобы модель была вотертай изначально!

In [3]:
def Fix(mesh): # фиксы проблем с нормалями и остальным (как будто по ракете кто-то стрелял). Возможно, придётся применять на каждом шагу 
    trimesh.repair.fix_inversion(mesh)
    trimesh.repair.fix_normals(mesh)
    trimesh.repair.fix_winding(mesh)

In [4]:
def save_to_obj(path, out_path):
    mesh = trimesh.load(path)
    trimesh.exchange.export.export_mesh(mesh, f'{out_path}/obj_correct_rocket.obj.', 'obj')

In [5]:
def load_trimesh_obj(path: Union[str, os.PathLike]) -> trimesh.Trimesh: #загружает файл из папки и в файле не один объект, а несколько, то он в сцене пытается объеденить объекты в одно множество. А дальше сохраняет в переменную obj геометрию для дальнейшей работы
    obj = trimesh.load(path)
    if type(obj) == trimesh.base.Trimesh:
        return obj
    elif type(obj) == trimesh.scene.scene.Scene:
        yourList = obj.geometry.items()
        vertice_list = [mesh.vertices for _, mesh in yourList]
        faces_list = [mesh.faces for _, mesh in yourList]
        faces_offset = np.cumsum([v.shape[0] for v in vertice_list])
        faces_offset = np.insert(faces_offset, 0, 0)[:-1]

        vertices = np.vstack(vertice_list)
        faces = np.vstack([face + offset for face, offset in zip(faces_list, faces_offset)])

        merged__meshes = trimesh.Trimesh(vertices, faces)
        return merged__meshes
    else:
        raise Exception('Unknown type of extension!')

In [6]:
def union_itself(mesh: trimesh.base.Trimesh) -> trimesh.base.Trimesh: # МАКСОН ТОПОВЫЙ ЧЕЛ!!!!!!! Работа с пересечением + разделение
    connected_groups = mesh.split(only_watertight= False)
    verts = [m.vertices for m in connected_groups if m.is_watertight]
    tris = [m.faces for m in connected_groups if m.is_watertight]
    for m in connected_groups:
        a = 0
        if not(m.is_watertight):
            
            trimesh.exchange.export.export_mesh(m, 'union_itself\\union_itself_{}.obj'.format(a), 'obj')
            a += 1
    vert_tmp, tris_tmp = pycork.union(verts[0], tris[0], 
                                         verts[1], tris[1])
    
    if len(connected_groups) == 2:
        mesh_res = trimesh.Trimesh(vertices=vert_tmp, faces=tris_tmp, process=True)
        
    elif len(connected_groups) > 2:
        for v, t in zip(verts[2:], tris[2:]):
            vert_tmp, tris_tmp = pycork.union(vert_tmp, tris_tmp, v, t)
        
        mesh_res = trimesh.Trimesh(vertices=vert_tmp, faces=tris_tmp, process=True)

    trimesh.exchange.export.export_mesh(mesh_res, 'union_itself\\union_itself_{}.obj'.format(a + 1), 'obj')

In [7]:
def manifold(path, out_path):
    list_obj = os.listdir(f'{path}')
    for i in range(len(list_obj)):
        subprocess.run(f"manifold.exe --input {path}/{list_obj[i]} --output {out_path}/manifold_{i}.obj --depth 8", shell=True)

In [8]:
def pyacvd_process(path, out_path): #работает на 100%
    list_obj = os.listdir(f'{path}')
    for i in range(len(list_obj)):
        rocket = pyvista.PolyData(f'{path}/{list_obj[i]}')
        clus = pyacvd.Clustering(rocket)
        clus.subdivide(3)
        clus.cluster(20000)
        remesh = clus.create_mesh()
        pyvista.save_meshio(f'{out_path}/pyacvd_{i}.obj', remesh, file_format='obj') 

In [9]:
def smoothing(path, out_path): #работает на 100%
    list_obj = os.listdir(f'{path}')
    for i in range(len(list_obj)):
        for _ in range(10):
            mesh = trimesh.load(f'{path}/{list_obj[i]}')
            Fix(mesh)
            smoothing_mesh = trimesh.smoothing.filter_humphrey(mesh)
            trimesh.exchange.export.export_mesh(smoothing_mesh, f'{out_path}/final_mesh{i}.obj', 'obj')

In [10]:
def assembling(path, out_path): # объединение всего в один объект, то есть в сцене объединяются все объекты воедино
    final_object = trimesh.scene.scene.Scene()
    list_obj = os.listdir(f'{path}')
    for i in range(len(list_obj)):
        mesh = trimesh.load_mesh(f'{path}/{list_obj[i]}')
        Fix(mesh)
        final_object.add_geometry(mesh)
    trimesh.exchange.export.export_scene(final_object, f'{out_path}/assembling.obj', 'obj')


In [11]:
def main(path, out_path):
    save_to_obj(path, "C:/Users/USER/Desktop/staja/save_to_obj")
    input_obj = load_trimesh_obj("C:/Users/USER/Desktop/staja/save_to_obj/obj_correct_rocket.obj")
    union_itself(input_obj)
    manifold("C:/Users/USER/Desktop/staja/union_itself", "C:/Users/USER/Desktop/staja/manifold")
    assembling("C:/Users/USER/Desktop/staja/manifold", "C:/Users/USER/Desktop/staja/assembling")
    pyacvd_process("C:/Users/USER/Desktop/staja/assembling", "C:/Users/USER/Desktop/staja/pyacvd") # работает на 100%
    smoothing("C:/Users/USER/Desktop/staja/pyacvd", out_path) # работает на 100%
        
if __name__ == '__main__':
    main(path = "C:/Users/USER/Desktop/staja/input/correct_rocket.stl", out_path = "C:/Users/USER/Desktop/staja/output")